## Carga librerias necesarias


In [1]:
import pandas as pd 
import numpy as np
import os
from pandas import concat

## Carga del los dataset correspondientes al año 2022
Se cargan los csv de cada mes en una lista

In [2]:
file_names = os.listdir('../data/anio22/')
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [3]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/anio22/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
    

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


## Filtrar los datos con el componente NO2

In [4]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 9)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
270,28,79,8,9,28079008_9_47,2022,4,1,4.0,V,...,8.0,V,4.0,V,5.0,V,10.0,V,5.0,V
271,28,79,8,9,28079008_9_47,2022,4,2,3.0,V,...,5.0,V,3.0,V,3.0,V,5.0,V,5.0,V
272,28,79,8,9,28079008_9_47,2022,4,3,6.0,V,...,1.0,V,1.0,V,5.0,V,4.0,V,3.0,V
273,28,79,8,9,28079008_9_47,2022,4,4,1.0,V,...,3.0,V,10.0,V,10.0,V,11.0,V,3.0,V
274,28,79,8,9,28079008_9_47,2022,4,5,1.0,V,...,11.0,V,7.0,V,8.0,V,4.0,V,1.0,V


## Crear una nueva columna fecha 

In [5]:
df_aire['fecha'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Extraer las datos de las medidas tomadas

In [6]:
data_pm2_5_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

## Crear un dataset auxiliar con los datos extraidos  y las nuevas columnas creadas

In [7]:


data_pm2_5 = pd.DataFrame(columns=['hora', 'dato', 'verificado'])
aux = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_pm2_5_columns, veri_columns): 
     aux['dato'] = df_aire [ data_pm2_5_columns[h-1] ] 
     aux['verificado'] = df_aire [ veri_columns[h-1] ] 
     aux['hora'] = h-1
     aux[['ano', 'mes', 'dia']] = df_aire[['ANO','MES','DIA']]
     aux['estacion'] = df_aire ['ESTACION']
     data_pm2_5 = pd.concat([data_pm2_5, aux], ignore_index=True)
data_pm2_5 = data_pm2_5.astype({'hora': 'int32'}, copy = False)

data_pm2_5.head()



,hora,dato,verificado,ano,mes,dia,estacion
0,0,4.0,V,2022.0,4.0,1.0,8.0
1,0,3.0,V,2022.0,4.0,2.0,8.0
2,0,6.0,V,2022.0,4.0,3.0,8.0
3,0,1.0,V,2022.0,4.0,4.0,8.0
4,0,1.0,V,2022.0,4.0,5.0,8.0


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [8]:
data_pm2_5['fecha'] = pd.to_datetime(data_pm2_5[['ano','mes','dia','hora']].rename(columns = {'ano': 'year', 'mes': 'month','dia': 'day', 'hora': 'hour'}))
data_pm2_5.head()

,hora,dato,verificado,ano,mes,dia,estacion,fecha
0,0,4.0,V,2022.0,4.0,1.0,8.0,2022-04-01
1,0,3.0,V,2022.0,4.0,2.0,8.0,2022-04-02
2,0,6.0,V,2022.0,4.0,3.0,8.0,2022-04-03
3,0,1.0,V,2022.0,4.0,4.0,8.0,2022-04-04
4,0,1.0,V,2022.0,4.0,5.0,8.0,2022-04-05


## Multi-index data frame con estacion y fecha :O

In [9]:
data_pm2_5.set_index(['estacion', 'fecha'], inplace=True)
data_pm2_5.head()

hora  dato verificado     ano  mes  dia
estacion fecha                                              
8.0      2022-04-01     0   4.0          V  2022.0  4.0  1.0
         2022-04-02     0   3.0          V  2022.0  4.0  2.0
         2022-04-03     0   6.0          V  2022.0  4.0  3.0
         2022-04-04     0   1.0          V  2022.0  4.0  4.0
         2022-04-05     0   1.0          V  2022.0  4.0  5.0

## Ordenar por estación y fecha 

In [10]:
data_pm2_5 = data_pm2_5.sort_values(['estacion', 'fecha'])
data_pm2_5.head()



hora  dato verificado     ano  mes  dia
estacion fecha                                                       
8.0      2022-01-01 00:00:00     0  15.0          V  2022.0  1.0  1.0
         2022-01-01 01:00:00     1  25.0          V  2022.0  1.0  1.0
         2022-01-01 02:00:00     2  25.0          V  2022.0  1.0  1.0
         2022-01-01 03:00:00     3  15.0          V  2022.0  1.0  1.0
         2022-01-01 04:00:00     4   3.0          V  2022.0  1.0  1.0

## Rellenar con NaN las filas perdidas 
 Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas


In [11]:
estaciones = data_pm2_5.index.get_level_values('estacion').unique()
datetime_index = pd.date_range(start='1/1/2022', end='12/31/2022 23:00:00', freq='H', name='fecha')
data_pm2_5_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_pm2_5_estacion = data_pm2_5[data_pm2_5['estacion'] == estacion]
    data_pm2_5_estacion = data_pm2_5.loc[estacion]
    print(estacion, len(data_pm2_5_estacion))
    data_pm2_5_estacion = data_pm2_5_estacion.reindex(datetime_index)
    data_pm2_5_estacion.reset_index(inplace=True)
    print(estacion, len(data_pm2_5_estacion))
    data_pm2_5_estacion['estacion'] = estacion
    data_pm2_5_nuevo = pd.concat([data_pm2_5_nuevo, data_pm2_5_estacion],ignore_index=True)
    
print(len(data_pm2_5_nuevo))

8.0 8760
8.0 8760
24.0 8736
24.0 8760
38.0 8760
38.0 8760
47.0 8688
47.0 8760
48.0 8760
48.0 8760
50.0 8664
50.0 8760
56.0 8760
56.0 8760
57.0 8760
57.0 8760
70080


In [12]:
# Multi-index data_pm2_5 frame con estacion y fecha :O
data_pm2_5_nuevo.set_index(['estacion', 'fecha'], inplace=True)

##  Actualizar  data con el nuevo df:

In [13]:
data_pm2_5 = data_pm2_5_nuevo
data_pm2_5.info()
data_pm2_5

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 70080 entries, (8.0, Timestamp('2022-01-01 00:00:00')) to (57.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   hora        69888 non-null  float64
 1   dato        69888 non-null  float64
 2   verificado  69888 non-null  object 
 3   ano         69888 non-null  float64
 4   mes         69888 non-null  float64
 5   dia         69888 non-null  float64
dtypes: float64(5), object(1)
memory usage: 3.7+ MB


hora  dato verificado     ano   mes   dia
estacion fecha                                                         
8.0      2022-01-01 00:00:00   0.0  15.0          V  2022.0   1.0   1.0
         2022-01-01 01:00:00   1.0  25.0          V  2022.0   1.0   1.0
         2022-01-01 02:00:00   2.0  25.0          V  2022.0   1.0   1.0
         2022-01-01 03:00:00   3.0  15.0          V  2022.0   1.0   1.0
         2022-01-01 04:00:00   4.0   3.0          V  2022.0   1.0   1.0
...                            ...   ...        ...     ...   ...   ...
57.0     2022-12-31 19:00:00  19.0  18.0          V  2022.0  12.0  31.0
         2022-12-31 20:00:00  20.0  23.0          V  2022.0  12.0  31.0
         2022-12-31 21:00:00  21.0  27.0          V  2022.0  12.0  31.0
         2022-12-31 22:00:00  22.0  23.0          V  2022.0  12.0  31.0
         2022-12-31 23:00:00  23.0  20.0          V  2022.0  12.0  31.0

[70080 rows x 6 columns]

In [14]:
# Mostramos los datos que se habían perdidos
missing_data_pm2_5 = data_pm2_5[data_pm2_5['dato'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data_pm2_5)}')
missing_data_pm2_5

Cantidad de datos no existentes (perdidos): 192


hora  dato verificado  ano  mes  dia
estacion fecha                                                    
24.0     2022-04-17 00:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 01:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 02:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 03:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-04-17 04:00:00   NaN   NaN        NaN  NaN  NaN  NaN
...                            ...   ...        ...  ...  ...  ...
50.0     2022-08-15 19:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-08-15 20:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-08-15 21:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-08-15 22:00:00   NaN   NaN        NaN  NaN  NaN  NaN
         2022-08-15 23:00:00   NaN   NaN        NaN  NaN  NaN  NaN

[192 rows x 6 columns]

In [15]:
# Rellenamos las columnas conciernientes a la fecha que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
dt = missing_data_pm2_5.index.get_level_values('fecha')
data_pm2_5.loc[data_pm2_5['dato'].isna(),'ano'] = dt.year
data_pm2_5.loc[data_pm2_5['dato'].isna(),'mes'] = dt.month
data_pm2_5.loc[data_pm2_5['dato'].isna(),'dia'] = dt.day
data_pm2_5.loc[data_pm2_5['dato'].isna(),'hora'] = dt.hour
data_pm2_5 = data_pm2_5.astype({'hora': 'int32'}, copy = False)

#data_pm2_5.loc[data_pm2_5['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
data_pm2_5[data_pm2_5['dato'].isna()]

hora  dato verificado     ano  mes   dia
estacion fecha                                                        
24.0     2022-04-17 00:00:00     0   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 01:00:00     1   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 02:00:00     2   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 03:00:00     3   NaN        NaN  2022.0  4.0  17.0
         2022-04-17 04:00:00     4   NaN        NaN  2022.0  4.0  17.0
...                            ...   ...        ...     ...  ...   ...
50.0     2022-08-15 19:00:00    19   NaN        NaN  2022.0  8.0  15.0
         2022-08-15 20:00:00    20   NaN        NaN  2022.0  8.0  15.0
         2022-08-15 21:00:00    21   NaN        NaN  2022.0  8.0  15.0
         2022-08-15 22:00:00    22   NaN        NaN  2022.0  8.0  15.0
         2022-08-15 23:00:00    23   NaN        NaN  2022.0  8.0  15.0

[192 rows x 6 columns]

In [16]:


# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data_pm2_5.loc[data_pm2_5['verificado'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data_pm2_5.loc[data_pm2_5['verificado'] != 'V', ['dato']] = np.nan
data_pm2_5.loc[data_pm2_5['verificado'] != 'V']



Cantidad de datos existentes pero no válidos:4340


hora  dato verificado     ano   mes   dia
estacion fecha                                                         
8.0      2022-01-09 18:00:00    18   NaN          N  2022.0   1.0   9.0
         2022-01-10 13:00:00    13   NaN          N  2022.0   1.0  10.0
         2022-01-10 14:00:00    14   NaN          N  2022.0   1.0  10.0
         2022-01-10 15:00:00    15   NaN          N  2022.0   1.0  10.0
         2022-01-14 11:00:00    11   NaN          N  2022.0   1.0  14.0
...                            ...   ...        ...     ...   ...   ...
57.0     2022-10-20 12:00:00    12   NaN          N  2022.0  10.0  20.0
         2022-11-11 13:00:00    13   NaN          N  2022.0  11.0  11.0
         2022-11-11 14:00:00    14   NaN          N  2022.0  11.0  11.0
         2022-11-11 15:00:00    15   NaN          N  2022.0  11.0  11.0
         2022-11-11 16:00:00    16   NaN          N  2022.0  11.0  11.0

[4340 rows x 6 columns]

In [17]:
# borramos columna verificado
del data_pm2_5['verificado']
data_pm2_5.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 70080 entries, (8.0, Timestamp('2022-01-01 00:00:00')) to (57.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hora    70080 non-null  int32  
 1   dato    65740 non-null  float64
 2   ano     70080 non-null  float64
 3   mes     70080 non-null  float64
 4   dia     70080 non-null  float64
dtypes: float64(4), int32(1)
memory usage: 2.9 MB


In [18]:


# Primero de todo, deshacemos el multíndice
data_pm2_5.reset_index(inplace=True) # Salvar los datos sin indices de estacion y fecha



In [19]:


# 7. Añadimos columna para el día de la semana
#dates = data_pm2_5.index.get_level_values('fecha').to_pydatetime() # con multiindex
dates = data_pm2_5['fecha'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_pm2_5['dia_semana'] = np.array([days_of_week[d.weekday()] for d in dates])



In [20]:
data_pm2_5 = data_pm2_5.rename(columns={'dato': 'PM2.5'}, copy = False)

In [21]:


data_pm2_5.info()
data_pm2_5.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70080 entries, 0 to 70079
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   estacion    70080 non-null  float64       
 1   fecha       70080 non-null  datetime64[ns]
 2   hora        70080 non-null  int32         
 3   PM2.5       65740 non-null  float64       
 4   ano         70080 non-null  float64       
 5   mes         70080 non-null  float64       
 6   dia         70080 non-null  float64       
 7   dia_semana  70080 non-null  object        
dtypes: datetime64[ns](1), float64(5), int32(1), object(1)
memory usage: 4.0+ MB


,estacion,fecha,hora,PM2.5,ano,mes,dia,dia_semana
0,8.0,2022-01-01 00:00:00,0,15.0,2022.0,1.0,1.0,Sabado
1,8.0,2022-01-01 01:00:00,1,25.0,2022.0,1.0,1.0,Sabado
2,8.0,2022-01-01 02:00:00,2,25.0,2022.0,1.0,1.0,Sabado
3,8.0,2022-01-01 03:00:00,3,15.0,2022.0,1.0,1.0,Sabado
4,8.0,2022-01-01 04:00:00,4,3.0,2022.0,1.0,1.0,Sabado


In [22]:
data_pm2_5.tail()

,estacion,fecha,hora,PM2.5,ano,mes,dia,dia_semana
70075,57.0,2022-12-31 19:00:00,19,18.0,2022.0,12.0,31.0,Sabado
70076,57.0,2022-12-31 20:00:00,20,23.0,2022.0,12.0,31.0,Sabado
70077,57.0,2022-12-31 21:00:00,21,27.0,2022.0,12.0,31.0,Sabado
70078,57.0,2022-12-31 22:00:00,22,23.0,2022.0,12.0,31.0,Sabado
70079,57.0,2022-12-31 23:00:00,23,20.0,2022.0,12.0,31.0,Sabado


In [23]:
# Serie con solamente los datos de contaminación
datos = data_pm2_5['PM2.5']
datos

0        15.0
1        25.0
2        25.0
3        15.0
4         3.0
         ... 
70075    18.0
70076    23.0
70077    27.0
70078    23.0
70079    20.0
Name: PM2.5, Length: 70080, dtype: float64

In [24]:
#datos.to_csv('data_pm2_5_hist.csv') # Salvado serie solo datos históricos
data_pm2_5.to_csv('../data/aire/data_pm2_5_fecha.csv') # Salvado con los datos de las fechas